In [1]:
from __future__ import division
import numpy as np
import scipy.linalg as slg
import math
import sys
from numpy import genfromtxt
import csv
from numpy import linalg as LA
from numpy.linalg import inv

In [2]:
def str2float(s):
    return float(s.replace('D','e'))

def CutLTMatOut(fname, head_line, index=0):
    cur_index = 0
    f = open(fname)
    status = None
    r = 0
    indices = []
    mat = []
    for line in f:
        if head_line in line:
            status = "Column Indices"
            continue
        if status == "Rows":
            splitted = line.split()
            if all([x.isnumeric() for x in splitted]):
                status = "Column Indices"
            else:
                try:
                    r = int(splitted[0])
                    floats = [str2float(s) for s in splitted[1:]]
                    if len(mat) < r:
                        mat.append(floats)
                    else:
                        mat[r-1] += floats
                except ValueError as e:
                    break
                continue
        if status == "Column Indices":
            status = "Rows"
            indices = [int(I) for I in line.split()]
            continue
    f.close()
    return mat

def toNPMatrix(mat_array, symmetry = 'symmetric'):
    dim = len(mat_array)
    mat = np.zeros((dim,dim))
    for i in range(dim):
        mat[i,:i+1] = mat_array[i]
    if symmetry == 'symmetric':
        diag = np.diag(np.diag(mat))
        return mat + mat.T - diag
    elif symmetry == 'anti-symmetric':
        return mat - mat.T


def ReadMatrix(fname, head_line, index=0):
    return np.array(CutLTMatOut(fname, head_line, index))

def ReadLowerTri(fname, head_line, index=0, symmetry = 'symmetric'):
    return toNPMatrix(CutLTMatOut(fname, head_line, index), symmetry)

def outMatrix(A):
    if np.iscomplexobj(A):
        print("Real part:")
        outMatrix(A.real)
        print()
        print("Imag part:")
        outMatrix(A.imag)
        return
    print("--------------------------------------------------------------------------------")
    curColStart, curColEnd = 0, 5
    while curColStart < A.shape[1]:
        curColEnd = min(curColStart + 5, A.shape[1])
        print("     "+"               ".join(list(map(str, range(curColStart + 1, curColEnd + 1)))))
        for i in range(A.shape[0]):
            print("%3d   "%(i+1,) + " ".join(list(map(lambda x : "%15.8e"%(x if abs(x) > 1e-8 else 0,) , A[i, curColStart : curColEnd]))))
        curColStart += 5
        print()

    print("--------------------------------------------------------------------------------")

def RMSD(V, W):
    """
    Calculate Root-mean-square deviation from two sets of vectors V and W.
    Parameters
    ----------
    V : array
        (N,D) matrix, where N is points and D is dimension.
    W : array
        (N,D) matrix, where N is points and D is dimension.
    Returns
    -------
    rmsd : float
        Root-mean-square deviation between the two vectors
    """
    diff = np.array(V) - np.array(W)
    N = len(V)
    return np.sqrt((diff * diff).sum() / N)
    

In [4]:
fname_fulllog = "./neo_small_basis_stw.log"
fname = "./neo_first_protonic_loop.log"
prot_S_headline = 'Prot S Matrix:          0:'
prot_V_headline = 'V for IR=Prot          1'

In [5]:
prot_S = ReadLowerTri(fname_fulllog, prot_S_headline)
prot_V = ReadMatrix(fname_fulllog, prot_V_headline)

In [6]:
print("Protonic Overlap Matrix: ")
outMatrix(prot_S)

Protonic Overlap Matrix: 
--------------------------------------------------------------------------------
     1               2               3               4               5
  1    1.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  2    0.00000000e+00  1.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  3    0.00000000e+00  0.00000000e+00  1.00000000e+00  0.00000000e+00  0.00000000e+00
  4    0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00  0.00000000e+00
  5    0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00
  6    0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  7    0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  8    0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00

     6               7               8
  1    0.00000000e+00  0.00000000e+00  0.00000000e+00
  2    0.00000000e+00  0.00000000e+00  0

In [7]:
print("Protonic Transformation Matrix: ")
outMatrix(prot_V)

Protonic Transformation Matrix: 
--------------------------------------------------------------------------------
     1               2               3               4               5
  1   -1.00971000e-01  0.00000000e+00  0.00000000e+00  7.00324000e-01  6.99397000e-01
  2    6.99861000e-01  0.00000000e+00  0.00000000e+00  1.01038000e-01  1.00904000e-01
  3    0.00000000e+00  9.08092000e-04  7.07107000e-01 -7.72592000e-05  7.73615000e-05
  4    0.00000000e+00  7.07106000e-01 -9.08119000e-04  0.00000000e+00  0.00000000e+00
  5   -1.00971000e-01  0.00000000e+00 -7.65487000e-05 -6.99397000e-01  7.00324000e-01
  6   -6.99861000e-01  0.00000000e+00  1.08274000e-05  1.00904000e-01 -1.01038000e-01
  7    0.00000000e+00  9.05093000e-04  7.07106000e-01  0.00000000e+00  0.00000000e+00
  8    0.00000000e+00  7.07106000e-01 -9.05067000e-04  0.00000000e+00  0.00000000e+00

     6               7               8
  1    0.00000000e+00  0.00000000e+00 -1.00971000e-01
  2    0.00000000e+00  0.00000000

In [8]:
Sigma_prot_S, L_prot_S  = LA.eig(prot_S)
prot_Sigma_minusOneHalf = (np.diag(Sigma_prot_S**(-0.5)) )
prot_V_manual = np.dot(L_prot_S, np.dot(prot_Sigma_minusOneHalf, np.transpose(L_prot_S)))

print("Manually calculated Protonic Transfromation Matrix V: ")
outMatrix(prot_V_manual)

Manually calculated Protonic Transfromation Matrix V: 
--------------------------------------------------------------------------------
     1               2               3               4               5
  1    1.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  2    0.00000000e+00  1.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  3    0.00000000e+00  0.00000000e+00  1.00000000e+00  0.00000000e+00  0.00000000e+00
  4    0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00  0.00000000e+00
  5    0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00
  6    0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  7    0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  8    0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00

     6               7               8
  1    0.00000000e+00  0.00000000e+00  0.00000000e+00
  2    0.00

In [9]:
iden = np.dot(np.dot(prot_V_manual.transpose(), prot_S), prot_V_manual)

print("Protonic V+SV, Should be identity: ")
outMatrix(iden)

Protonic V+SV, Should be identity: 
--------------------------------------------------------------------------------
     1               2               3               4               5
  1    1.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  2    0.00000000e+00  1.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  3    0.00000000e+00  0.00000000e+00  1.00000000e+00  0.00000000e+00  0.00000000e+00
  4    0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00  0.00000000e+00
  5    0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00
  6    0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  7    0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  8    0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00

     6               7               8
  1    0.00000000e+00  0.00000000e+00  0.00000000e+00
  2    0.00000000e+00  0.00000

In [10]:
prot_V_RMSD = RMSD(prot_V, prot_V_manual)
print(prot_V_RMSD)

1.3602066491621623


In [11]:
Fp1_headline = 'Fp1 from l501:          0:'
Fp1 = ReadLowerTri(fname, Fp1_headline)
Pp1_headline = 'Protonic Density1:          0:'
Pp1 = ReadLowerTri(fname, Pp1_headline)
e1_headline = 'Electronic Alpha error matrix at iteration1:'
e1 = ReadLowerTri(fname, e1_headline)

In [12]:
outMatrix(e1)

--------------------------------------------------------------------------------
     1               2               3               4               5
  1    0.00000000e+00 -8.44500000e-03  0.00000000e+00  4.11372000e-03  0.00000000e+00
  2   -8.44500000e-03  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  3    0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  4    4.11372000e-03  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  5    0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  6    0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  8.44500000e-03
  7    0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  8    0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  4.11372000e-03

     6               7               8
  1    0.00000000e+00  0.00000000e+00  0.00000000e+00
  2    0.00000000e+00  0.00000000e+00  0.00000000e+00
  3    0.000

In [13]:
outMatrix(Fp1)

--------------------------------------------------------------------------------
     1               2               3               4               5
  1   -9.33881000e-01 -1.68900000e-02  0.00000000e+00  8.22743000e-03  0.00000000e+00
  2   -1.68900000e-02  6.43728000e-01  0.00000000e+00 -1.53680000e-02  0.00000000e+00
  3    0.00000000e+00  0.00000000e+00  6.17491000e-01  0.00000000e+00  0.00000000e+00
  4    8.22743000e-03 -1.53680000e-02  0.00000000e+00  6.26318000e-01  0.00000000e+00
  5    0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00 -9.33881000e-01
  6    0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  1.68900000e-02
  7    0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  8    0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  8.22743000e-03

     6               7               8
  1    0.00000000e+00  0.00000000e+00  0.00000000e+00
  2    0.00000000e+00  0.00000000e+00  0.00000000e+00
  3    0.000

In [14]:
outMatrix(Pp1)

--------------------------------------------------------------------------------
     1               2               3               4               5
  1    1.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  2    0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  3    0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  4    0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  5    0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00
  6    0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  7    0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  8    0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00

     6               7               8
  1    0.00000000e+00  0.00000000e+00  0.00000000e+00
  2    0.00000000e+00  0.00000000e+00  0.00000000e+00
  3    0.000

In [15]:
FP_manual = np.dot(Fp1, Pp1)
outMatrix(FP_manual)

--------------------------------------------------------------------------------
     1               2               3               4               5
  1   -9.33881000e-01  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  2   -1.68900000e-02  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  3    0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  4    8.22743000e-03  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  5    0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00 -9.33881000e-01
  6    0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  1.68900000e-02
  7    0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  8    0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  8.22743000e-03

     6               7               8
  1    0.00000000e+00  0.00000000e+00  0.00000000e+00
  2    0.00000000e+00  0.00000000e+00  0.00000000e+00
  3    0.000

In [16]:
outMatrix(prot_S)

--------------------------------------------------------------------------------
     1               2               3               4               5
  1    1.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  2    0.00000000e+00  1.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  3    0.00000000e+00  0.00000000e+00  1.00000000e+00  0.00000000e+00  0.00000000e+00
  4    0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00  0.00000000e+00
  5    0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00
  6    0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  7    0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  8    0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00

     6               7               8
  1    0.00000000e+00  0.00000000e+00  0.00000000e+00
  2    0.00000000e+00  0.00000000e+00  0.00000000e+00
  3    0.000

In [18]:
FPS_manual = np.dot(np.dot(Fp1, Pp1), prot_S)
outMatrix(FPS_manual)

--------------------------------------------------------------------------------
     1               2               3               4               5
  1   -9.33881000e-01  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  2   -1.68900000e-02  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  3    0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  4    8.22743000e-03  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  5    0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00 -9.33881000e-01
  6    0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  1.68900000e-02
  7    0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  8    0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  8.22743000e-03

     6               7               8
  1    0.00000000e+00  0.00000000e+00  0.00000000e+00
  2    0.00000000e+00  0.00000000e+00  0.00000000e+00
  3    0.000

In [19]:
e1_manual_AO = FPS_manual - FPS_manual.transpose()
outMatrix(e1_manual_AO)

--------------------------------------------------------------------------------
     1               2               3               4               5
  1    0.00000000e+00  1.68900000e-02  0.00000000e+00 -8.22743000e-03  0.00000000e+00
  2   -1.68900000e-02  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  3    0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  4    8.22743000e-03  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  5    0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  6    0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  1.68900000e-02
  7    0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  8    0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  8.22743000e-03

     6               7               8
  1    0.00000000e+00  0.00000000e+00  0.00000000e+00
  2    0.00000000e+00  0.00000000e+00  0.00000000e+00
  3    0.000

In [20]:
outMatrix(prot_V)

--------------------------------------------------------------------------------
     1               2               3               4               5
  1   -1.00971000e-01  0.00000000e+00  0.00000000e+00  7.00324000e-01  6.99397000e-01
  2    6.99861000e-01  0.00000000e+00  0.00000000e+00  1.01038000e-01  1.00904000e-01
  3    0.00000000e+00  9.08092000e-04  7.07107000e-01 -7.72592000e-05  7.73615000e-05
  4    0.00000000e+00  7.07106000e-01 -9.08119000e-04  0.00000000e+00  0.00000000e+00
  5   -1.00971000e-01  0.00000000e+00 -7.65487000e-05 -6.99397000e-01  7.00324000e-01
  6   -6.99861000e-01  0.00000000e+00  1.08274000e-05  1.00904000e-01 -1.01038000e-01
  7    0.00000000e+00  9.05093000e-04  7.07106000e-01  0.00000000e+00  0.00000000e+00
  8    0.00000000e+00  7.07106000e-01 -9.05067000e-04  0.00000000e+00  0.00000000e+00

     6               7               8
  1    0.00000000e+00  0.00000000e+00 -1.00971000e-01
  2    0.00000000e+00  0.00000000e+00  6.99861000e-01
  3   -8.859

In [21]:
# FPS-SPF
e1_manual_ON = np.dot(prot_V.transpose(), np.dot(e1_manual_AO, prot_V))
print("Error Matrix (in orthonormal basis):")
outMatrix(e1_manual_ON)

Error Matrix (in orthonormal basis):
--------------------------------------------------------------------------------
     1               2               3               4               5
  1    0.00000000e+00  1.17483093e-03 -5.82950716e-07 -1.11862683e-05 -1.68900072e-02
  2   -1.17483093e-03  0.00000000e+00 -4.45334702e-07  5.39297556e-06  8.14310804e-03
  3    5.82950716e-07  4.45334702e-07  0.00000000e+00 -2.19305411e-08 -1.04429711e-05
  4    1.11862683e-05 -5.39297556e-06  2.19305411e-08  0.00000000e+00  0.00000000e+00
  5    1.68900072e-02 -8.14310804e-03  1.04429711e-05  0.00000000e+00  0.00000000e+00
  6    0.00000000e+00  0.00000000e+00  4.45334702e-07  8.14310804e-03 -5.39297556e-06
  7   -9.20829731e-07  4.45334120e-07  0.00000000e+00  1.02226931e-05 -2.65864521e-08
  8    0.00000000e+00  0.00000000e+00 -9.25856010e-07 -1.68900072e-02  1.11862683e-05

     6               7               8
  1    0.00000000e+00  9.20829731e-07  0.00000000e+00
  2    0.00000000e+00 -4.4533

In [22]:
e1_manual_ON_0p5 = e1_manual_ON *0.5
outMatrix(e1_manual_ON_0p5)

--------------------------------------------------------------------------------
     1               2               3               4               5
  1    0.00000000e+00  5.87415465e-04 -2.91475358e-07 -5.59313415e-06 -8.44500358e-03
  2   -5.87415465e-04  0.00000000e+00 -2.22667351e-07  2.69648778e-06  4.07155402e-03
  3    2.91475358e-07  2.22667351e-07  0.00000000e+00 -1.09652705e-08 -5.22148557e-06
  4    5.59313415e-06 -2.69648778e-06  1.09652705e-08  0.00000000e+00  0.00000000e+00
  5    8.44500358e-03 -4.07155402e-03  5.22148557e-06  0.00000000e+00  0.00000000e+00
  6    0.00000000e+00  0.00000000e+00  2.22667351e-07  4.07155402e-03 -2.69648778e-06
  7   -4.60414865e-07  2.22667060e-07  0.00000000e+00  5.11134656e-06 -1.32932260e-08
  8    0.00000000e+00  0.00000000e+00 -4.62928005e-07 -8.44500358e-03  5.59313415e-06

     6               7               8
  1    0.00000000e+00  4.60414865e-07  0.00000000e+00
  2    0.00000000e+00 -2.22667060e-07  0.00000000e+00
  3   -2.226

In [23]:
print(np.trace(np.dot(e1_manual_ON_0p5,e1_manual_ON_0p5)))

-0.0003529630409417583


In [24]:
def getErrorMatrix(F,P,S,V,printlevel):
    FPS = np.dot(np.dot(F, P), S)

    e_AO = FPS - FPS.transpose()

    e_ON = np.dot(V.transpose(), np.dot(e_AO, V))

    if (printlevel >= 1):
        if (printlevel >= 2):
            print('F')
            outMatrix(F)
            print('P')
            outMatrix(P)
            print('FP')
            outMatrix(np.dot(F, P))
            print('S')   
            outMatrix(S)
        print('FPS')
        outMatrix(FPS)
        print('e_AO')
        outMatrix(e_AO)
        if (printlevel >= 2):
            print('V')
            outMatrix(V)
        print('e_ON')
        outMatrix(e_ON)
    return 0.5 * e_ON

In [25]:
def getBmatrix(E_list):
    Bsize = len(E_list)
    B = np.zeros((Bsize+1,Bsize+1))
    
    B[0,0] = 0.00
    for i in range(1,Bsize+1):
        B[i,0] = -1
        B[0,i] = -1
        B[i,i] = np.trace(np.dot(E_list[i-1],E_list[i-1]))
        for j in range(i+1,Bsize+1):
            B[i,j] = np.trace(np.dot(E_list[i-1],E_list[j-1]))
            B[j,i] = B[i,j]
        
    return B

In [26]:
def newF(F_list, B):
    Fsize = len(F_list)
    NBasis = F_list[0].shape[0]
    
    F = np.zeros((NBasis,NBasis))
    for i in range(Fsize):
        ci = inv(B)[:,0][1:][i] * (-1)
        print('c' + str(i) + ': ', ci)
        F += ci * F_list[i]
    return F

In [27]:
E_list = [e1_manual_ON_0p5]
F_list = [Fp1]
B = getBmatrix(E_list)
outMatrix(B)

--------------------------------------------------------------------------------
     1               2
  1    0.00000000e+00 -1.00000000e+00
  2   -1.00000000e+00 -3.52963041e-04

--------------------------------------------------------------------------------


In [28]:
newF_B1 = newF(F_list,B)
outMatrix(newF_B1)

c0:  1.0
--------------------------------------------------------------------------------
     1               2               3               4               5
  1   -9.33881000e-01 -1.68900000e-02  0.00000000e+00  8.22743000e-03  0.00000000e+00
  2   -1.68900000e-02  6.43728000e-01  0.00000000e+00 -1.53680000e-02  0.00000000e+00
  3    0.00000000e+00  0.00000000e+00  6.17491000e-01  0.00000000e+00  0.00000000e+00
  4    8.22743000e-03 -1.53680000e-02  0.00000000e+00  6.26318000e-01  0.00000000e+00
  5    0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00 -9.33881000e-01
  6    0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  1.68900000e-02
  7    0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  8    0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  8.22743000e-03

     6               7               8
  1    0.00000000e+00  0.00000000e+00  0.00000000e+00
  2    0.00000000e+00  0.00000000e+00  0.00000000e+00
  3

In [29]:
Fp2_headline = 'Fp2 from l501:          0:'
Fp2 = ReadLowerTri(fname, Fp2_headline)
Pp2_headline = 'Protonic Density2:          0:'
Pp2 = ReadLowerTri(fname, Pp2_headline)
e2_headline = 'Electronic Alpha error matrix at iteration for IMat=    2:'
e2 = ReadLowerTri(fname, e2_headline)
e2_manual_ON_0p5 = getErrorMatrix(Fp2,Pp2,prot_S,prot_V,0)

In [30]:
E_list.append(e2_manual_ON_0p5)
F_list.append(Fp2)
B = getBmatrix(E_list)
outMatrix(B)

--------------------------------------------------------------------------------
     1               2               3
  1    0.00000000e+00 -1.00000000e+00 -1.00000000e+00
  2   -1.00000000e+00 -3.52963041e-04 -3.33522484e-04
  3   -1.00000000e+00 -3.33522484e-04 -3.15152963e-04

--------------------------------------------------------------------------------


In [31]:
newF_B2 = newF(F_list,B)
outMatrix(newF_B2)

c0:  -17.15117178383393
c1:  18.15117178383393
--------------------------------------------------------------------------------
     1               2               3               4               5
  1   -9.37783502e-01 -3.06707945e-01  0.00000000e+00  1.49063642e-01  0.00000000e+00
  2   -3.06707945e-01  6.32673936e-01  0.00000000e+00 -1.38088143e-02  0.00000000e+00
  3    0.00000000e+00  0.00000000e+00  6.09885659e-01  0.00000000e+00  0.00000000e+00
  4    1.49063642e-01 -1.38088143e-02  0.00000000e+00  6.17986612e-01  0.00000000e+00
  5    0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00 -9.37783502e-01
  6    0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  3.06707945e-01
  7    0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  8    0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  1.49063642e-01

     6               7               8
  1    0.00000000e+00  0.00000000e+00  0.00000000e+00
  2    0.00000000e+

In [171]:
Fp3_headline = 'Fp3 from l501:          0:'
Fp3 = ReadLowerTri(fname, Fp3_headline)
Pp3_headline = 'Protonic Density3:          0:'
Pp3 = ReadLowerTri(fname, Pp3_headline)
e3_headline = 'Electronic Alpha error matrix at iteration for IMat=    3:'
e3 = ReadLowerTri(fname, e3_headline)
e3_manual_ON_0p5 = getErrorMatrix(Fp3,Pp3,prot_S,prot_V,0)

In [172]:
E_list.append(e3_manual_ON_0p5)
F_list.append(Fp3)
B = getBmatrix(E_list)
outMatrix(B)

--------------------------------------------------------------------------------
     1               2               3               4
  1    0.00000000e+00 -1.00000000e+00 -1.00000000e+00 -1.00000000e+00
  2   -1.00000000e+00 -3.52962704e-04 -3.33522166e-04 -2.58294832e-06
  3   -1.00000000e+00 -3.33522166e-04 -3.15152662e-04 -2.44570156e-06
  4   -1.00000000e+00 -2.58294832e-06 -2.44570156e-06 -1.11131053e-07

--------------------------------------------------------------------------------


In [173]:
RMSD(b3,B)

2.5598546369968736e-09

In [174]:
newF_B3 = newF(F_list,B)
outMatrix(newF_B3)

c0:  -7.4065337215474125
c1:  7.834095132431371
c2:  0.5724385891160413
--------------------------------------------------------------------------------
     1               2               3               4               5
  1   -9.04291293e-01 -2.95139600e-01  0.00000000e+00  1.43270355e-01  0.00000000e+00
  2   -2.95139600e-01  6.06181492e-01  0.00000000e+00 -8.20067178e-04  0.00000000e+00
  3    0.00000000e+00  0.00000000e+00  6.10227204e-01  0.00000000e+00  0.00000000e+00
  4    1.43270355e-01 -8.20067178e-04  0.00000000e+00  6.12040446e-01  0.00000000e+00
  5    0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00 -9.04291293e-01
  6    0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  2.95139600e-01
  7    0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  8    0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  1.43270355e-01

     6               7               8
  1    0.00000000e+00  0.00000000e+00  0.0000000

In [175]:
b3_headline = 'B matrix at iteration          3'
b3 = ReadMatrix(fname, b3_headline)
outMatrix(b3)

--------------------------------------------------------------------------------
     1               2               3               4
  1    0.00000000e+00 -1.00000000e+00 -1.00000000e+00 -1.00000000e+00
  2   -1.00000000e+00 -3.52962000e-04 -3.33520000e-04 -2.58208000e-06
  3   -1.00000000e+00 -3.33520000e-04 -3.15149000e-04 -2.44486000e-06
  4   -1.00000000e+00 -2.58208000e-06 -2.44486000e-06 -1.11071000e-07

--------------------------------------------------------------------------------


In [176]:
newF_b3 = newF(F_list,b3)
outMatrix(newF_b3)

c0:  0.5496322152994674
c1:  -0.5894416296165043
c2:  1.0398094143170369
--------------------------------------------------------------------------------
     1               2               3               4               5
  1   -8.76946362e-01 -2.85692971e-01  0.00000000e+00  1.38539621e-01  0.00000000e+00
  2   -2.85692971e-01  5.84551642e-01  0.00000000e+00  9.78466268e-03  0.00000000e+00
  3    0.00000000e+00  0.00000000e+00  6.10506102e-01  0.00000000e+00  0.00000000e+00
  4    1.38539621e-01  9.78466268e-03  0.00000000e+00  6.07185710e-01  0.00000000e+00
  5    0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00 -8.76946362e-01
  6    0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  2.85692971e-01
  7    0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  8    0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  1.38539621e-01

     6               7               8
  1    0.00000000e+00  0.00000000e+00  0.000000

In [177]:
Fp4_headline = 'Fp4 from l501:          0:'
Fp4 = ReadLowerTri(fname, Fp4_headline)
Pp4_headline = 'Protonic Density4:          0:'
Pp4 = ReadLowerTri(fname, Pp4_headline)
e4_headline = 'Electronic Alpha error matrix at iteration for IMat=    4:'
e4 = ReadLowerTri(fname, e4_headline)
e4_manual_ON_0p5 = getErrorMatrix(Fp4,Pp4,prot_S,prot_V,2)

F
--------------------------------------------------------------------------------
     1               2               3               4               5
  1   -8.78836000e-01 -2.85426000e-01  0.00000000e+00  1.38252000e-01  0.00000000e+00
  2   -2.85426000e-01  5.86067000e-01  0.00000000e+00  9.02246000e-03  0.00000000e+00
  3    0.00000000e+00  0.00000000e+00  6.10511000e-01  0.00000000e+00  0.00000000e+00
  4    1.38252000e-01  9.02246000e-03  0.00000000e+00  6.07575000e-01  0.00000000e+00
  5    0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00 -8.78836000e-01
  6    0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  2.85426000e-01
  7    0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  8    0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  1.38252000e-01

     6               7               8
  1    0.00000000e+00  0.00000000e+00  0.00000000e+00
  2    0.00000000e+00  0.00000000e+00  0.00000000e+00
  3    0.0

In [178]:
E_list.append(e4_manual_ON_0p5)
F_list.append(Fp4)
B = getBmatrix(E_list)
outMatrix(B)

--------------------------------------------------------------------------------
     1               2               3               4               5
  1    0.00000000e+00 -1.00000000e+00 -1.00000000e+00 -1.00000000e+00 -1.00000000e+00
  2   -1.00000000e+00 -3.52962704e-04 -3.33522166e-04 -2.58294832e-06 -1.30773715e-06
  3   -1.00000000e+00 -3.33522166e-04 -3.15152662e-04 -2.44570156e-06 -1.24064902e-06
  4   -1.00000000e+00 -2.58294832e-06 -2.44570156e-06 -1.11131053e-07 -1.00387751e-07
  5   -1.00000000e+00 -1.30773715e-06 -1.24064902e-06 -1.00387751e-07 -9.42732909e-08

--------------------------------------------------------------------------------


In [71]:
print('Determinant of B', np.linalg.det(B))

Determinant of B 1.478521294472216e-25


In [79]:
sigma_B, L_B  = LA.eig(B)

In [80]:
sigma_B

array([ 1.99983120e+00, -2.00016888e+00, -3.30640751e-04, -1.65139482e-11,
       -6.76955752e-12])

In [179]:
b4_headline = 'B matrix at iteration          4'
b4 = ReadMatrix(fname, b4_headline)

In [180]:
outMatrix(b4)

--------------------------------------------------------------------------------
     1               2               3               4               5
  1    0.00000000e+00 -1.00000000e+00 -1.00000000e+00 -1.00000000e+00 -1.00000000e+00
  2   -1.00000000e+00 -3.52962000e-04 -3.33520000e-04 -2.58208000e-06 -1.30565000e-06
  3   -1.00000000e+00 -3.33520000e-04 -3.15149000e-04 -2.44486000e-06 -1.23866000e-06
  4   -1.00000000e+00 -2.58208000e-06 -2.44486000e-06 -1.11071000e-07 -1.00408000e-07
  5   -1.00000000e+00 -1.30565000e-06 -1.23866000e-06 -1.00408000e-07 -9.43808000e-08

--------------------------------------------------------------------------------


In [181]:
print('Determinant of B', np.linalg.det(b4))

Determinant of B -1.9739028651611987e-24


In [182]:
sigma_b4, L_b4  = LA.eig(b4)

In [183]:
sigma_b4

array([ 1.99983120e+00, -2.00016887e+00, -3.30641582e-04, -1.53551240e-11,
        9.71974734e-11])

In [184]:
-112.136466031902685+112.136465554172347

-4.777303388436849e-07

In [48]:
Fp5_headline = 'Fp5 from l501:          0:'
Fp5 = ReadLowerTri(fname, Fp5_headline)
Pp5_headline = 'Protonic Density5:          0:'
Pp5 = ReadLowerTri(fname, Pp5_headline)
e5_headline = 'Electronic Alpha error matrix at iteration for IMat=    5:'
e5 = ReadLowerTri(fname, e5_headline)
e5_manual_ON_0p5 = getErrorMatrix(Fp5,Pp5,prot_S,prot_V,0)

In [49]:
E_list.append(e5_manual_ON_0p5)
B = getBmatrix(E_list)
outMatrix(B)

--------------------------------------------------------------------------------
     1               2               3               4               5
  1    0.00000000e+00 -1.00000000e+00 -1.00000000e+00 -1.00000000e+00 -1.00000000e+00
  2   -1.00000000e+00 -3.52962704e-04 -3.33522166e-04 -2.58294832e-06 -1.30773715e-06
  3   -1.00000000e+00 -3.33522166e-04 -3.15152662e-04 -2.44570156e-06 -1.24064902e-06
  4   -1.00000000e+00 -2.58294832e-06 -2.44570156e-06 -1.11131053e-07 -1.00387751e-07
  5   -1.00000000e+00 -1.30773715e-06 -1.24064902e-06 -1.00387751e-07 -9.42732909e-08
  6   -1.00000000e+00  3.23354586e-07  3.00855741e-07 -8.38661018e-08 -8.37148280e-08

     6
  1   -1.00000000e+00
  2    3.23354586e-07
  3    3.00855741e-07
  4   -8.38661018e-08
  5   -8.37148280e-08
  6   -8.09219549e-08

--------------------------------------------------------------------------------


In [50]:
print('Determinant of B', np.linalg.det(B))

Determinant of B 4.0789091368483385e-44


In [192]:
-5.05414064e-07+0.500992e-06

-4.422064000000005e-09

In [53]:
print('Determinant of B', np.linalg.det(b5))

Determinant of B -4.931156500005075e-19


In [54]:
outMatrix(b5)

--------------------------------------------------------------------------------
     1               2               3               4
  1    0.00000000e+00 -1.00000000e+00 -1.00000000e+00 -1.00000000e+00
  2   -1.00000000e+00 -1.11071000e-07 -1.00408000e-07 -8.36548000e-08
  3   -1.00000000e+00 -1.00408000e-07 -9.43808000e-08 -8.36027000e-08
  4   -1.00000000e+00 -8.36548000e-08 -8.36027000e-08 -8.06323000e-08

--------------------------------------------------------------------------------


--------------------------------------------------------------------------------
     1               2               3               4
  1    0.00000000e+00 -1.00000000e+00 -1.00000000e+00 -1.00000000e+00
  2   -1.00000000e+00 -3.52962000e-04 -3.33520000e-04 -2.58208000e-06
  3   -1.00000000e+00 -3.33520000e-04 -3.15149000e-04 -2.44486000e-06
  4   -1.00000000e+00 -2.58208000e-06 -2.44486000e-06 -1.11071000e-07

--------------------------------------------------------------------------------


In [148]:
newF_b3 = newF(F_list,b3)
outMatrix(newF_b3)

c0:  0.5496322152994674
c1:  -0.5894416296165043
c2:  1.0398094143170369
--------------------------------------------------------------------------------
     1               2               3               4               5
  1   -8.76946362e-01 -2.85692971e-01  0.00000000e+00  1.38539621e-01  0.00000000e+00
  2   -2.85692971e-01  5.84551642e-01  0.00000000e+00  9.78466268e-03  0.00000000e+00
  3    0.00000000e+00  0.00000000e+00  6.10506102e-01  0.00000000e+00  0.00000000e+00
  4    1.38539621e-01  9.78466268e-03  0.00000000e+00  6.07185710e-01  0.00000000e+00
  5    0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00 -8.76946362e-01
  6    0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  2.85692971e-01
  7    0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  8    0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  1.38539621e-01

     6               7               8
  1    0.00000000e+00  0.00000000e+00  0.000000

array([ 0.54963222, -0.58944163,  1.03980941])

In [186]:
E_list_reduce = [e3,e4]
B_reduce = getBmatrix(E_list_reduce )
outMatrix(B_reduce)

--------------------------------------------------------------------------------
     1               2               3
  1    0.00000000e+00 -1.00000000e+00 -1.00000000e+00
  2   -1.00000000e+00  1.11070752e-07  1.00408187e-07
  3   -1.00000000e+00  1.00408187e-07  9.43806824e-08

--------------------------------------------------------------------------------


In [187]:
F_list_reduce = [Fp3,Fp4]

In [188]:
newF_reduce = newF(F_list_reduce,B_reduce)
outMatrix(newF_reduce )

c0:  -1.3004158147816511
c1:  2.300415814781651
--------------------------------------------------------------------------------
     1               2               3               4               5
  1   -8.78300229e-01 -2.86691305e-01  0.00000000e+00  1.38646026e-01  0.00000000e+00
  2   -2.86691305e-01  5.85540332e-01  0.00000000e+00  9.22022724e-03  0.00000000e+00
  3    0.00000000e+00  0.00000000e+00  6.10478490e-01  0.00000000e+00  0.00000000e+00
  4    1.38646026e-01  9.22022724e-03  0.00000000e+00  6.07467065e-01  0.00000000e+00
  5    0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00 -8.78300229e-01
  6    0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  2.86691305e-01
  7    0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  8    0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  1.38646026e-01

     6               7               8
  1    0.00000000e+00  0.00000000e+00  0.00000000e+00
  2    0.00000000e